# Construct Comparison Dataset

In [1]:
import numpy as np
import json
import os

In [ ]:
if not os.path.exists('comparison_data'):
    os.makedirs('comparison_data')

## Read Data

In [3]:
d = './data'

with open(d+'/x_node_type.json', 'r') as fname:
    x_typ = json.load(fname)

            
with open(d+'/x_node_feature.json', 'r') as fname:
    x_ftr = json.load(fname)
        
        
with open(d+'/y_location.json', 'r') as fname:
    y_loc = json.load(fname)
    
    
with open(d+'/y_makespan.json', 'r') as fname:
    y_mkp = json.load(fname)

n = len(x_typ)

print("%g datapoints in original dataset" % n)


52881 datapoints in original dataset


## Construct a Non-sequential Dataset
**x**

- buffer level -- each buffer
- machine status -- each machine
- machine ratio -- each machine
- aggregated processing time -- each buffer
- product index
    
**y**
- makespan


Shuffling data with same seed in PCT main model

In [7]:
def shuffle(arr, N, seed=123):
    idx = np.arange(N)
    np.random.seed(seed)
    np.random.shuffle(idx)
    
    res = []
    for data in arr:
        res.append(data[idx])
    
    return res


In [8]:
N = len(x_ftr)

x_typ = np.array(x_typ)
x_ftr = np.array(x_ftr)
y_loc = np.array(y_loc)
y_mkp = np.array(y_mkp)

[x_ftr, x_typ, y_loc, y_mkp] = shuffle([x_ftr, x_typ, y_loc, y_mkp], N)

In [11]:
def data(x_typ, x_ftr, y_loc, y_mkp, name):
    x = []
    y_of_loc = []
    y_of_mkp = []

    #for i in range(len(x_typ)):
    for i in range(len(x_typ)):
        x_item = []

        # Buffer level
        bl = []
        b = 0
        # Machine status
        ms = []
        # Progress
        pg = []

        # Processing Time
        pt = np.zeros(8)
        ppt = []

        yy_loc = []
        yy_mkp = []
        q = 0
        for j in range(len(x_typ[i])):

            if x_typ[i][j] == [1]:

                q += 1

                pt += np.array(x_ftr[i][j][2:])
                # Product
                b += 1

                yy_loc.append(y_loc[i][j])
                yy_mkp.append(y_mkp[i][j])


            else:
                # Machine
                ms.append(x_ftr[i][j][0])
                pg.append(x_ftr[i][j][1])



                if j > 0:
                    bl.append(b)
                    b = 0
                    ppt = ppt + (pt.tolist())
                    pt = np.zeros(8)

        x_item = bl + ms + pg + ppt   

        y_of_loc += yy_loc
        y_of_mkp += yy_mkp


        for j in range(q):
            x.append(x_item + [j])

    x = np.array(x)
    y_of_loc = np.array(y_of_loc)
    y_of_mkp = np.array(y_of_mkp)


    with open('./comparison_data/'+ name + '_x.npy', 'wb') as fname:
        np.save(fname, x)

    with open('./comparison_data/' + name + '_y_loc.npy', 'wb') as fname:
        np.save(fname, y_of_loc)

    with open('./comparison_data/' + name + '_y_mkp.npy', 'wb') as fname:
        np.save(fname, y_of_mkp)
    
    print(name + ' data stored')

    

# Spliting Data
Split data with the same ratio (70:30) in main model

In [12]:
N_train = int(N * 0.7)

data(x_typ[:N_train], x_ftr[:N_train], y_loc[:N_train], y_mkp[:N_train], 'train')
data(x_typ[N_train:], x_ftr[N_train:], y_loc[N_train:], y_mkp[N_train:], 'test')

train data stored
test data stored
